In [1]:
# ==========================
# Install Required Libraries
# ==========================

# NLP and Text Preprocessing
!pip install nltk contractions inflect

!pip install gensim

# NLP libraries
import nltk
from nltk.corpus import stopwords
import gensim
from gensim.parsing.preprocessing import STOPWORDS as gensim_stopwords

# Data handling and numerical operations
!pip install pandas numpy

# ==========================
# Download necessary NLTK data
# ==========================
nltk.download('stopwords')
nltk.download('punkt')

# Basic libraries
import string
import pandas as pd
import numpy as np
import re
import contractions
import inflect






[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
import pandas as pd
import re
import contractions
import html

# Load once
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Bert/Data/new_articles.csv")

# --- Step 1: Lowercase ---
df['text'] = df['text'].astype(str).str.lower()

# --- Step 2: Expand contractions ---
df['text'] = df['text'].apply(contractions.fix)

# --- Step 3: Remove links ---
def remove_links(text):
    return re.sub(r'http\S+|www\S+|https\S+', '', text)
df['text'] = df['text'].apply(remove_links)

# --- Step 4: Remove HTML tags ---
def remove_html_tags(text):
    return re.sub(r'<.*?>', ' ', str(text))  # strip tags like <p>, <div>, etc.
df['text'] = df['text'].apply(remove_html_tags)
df['text'] = df['text'].apply(html.unescape)


# --- Step 5: Clean unicode + extra spaces ---
def clean_text(text):
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)  # remove non-ASCII chars
    text = re.sub(r'\s+', ' ', text)            # collapse whitespace
    return text.strip()
df['text'] = df['text'].apply(clean_text)

# --- Step 6: Ensure labels are binary ints ---
if df['label'].dtype == object:  # e.g., "Real"/"Fake"
    df['label'] = df['label'].map({'Real': 1, 'Fake': 0})
else:  # if already numeric but float
    df['label'] = df['label'].astype(int)

df = df.dropna(subset=["text", "label"])  # remove rows missing text/label

# --- Save final preprocessed dataset ---
out_path = "/content/drive/MyDrive/Colab Notebooks/Bert/Data/processed_new_articles.csv"
df.to_csv(out_path, index=False)

print("✅ Preprocessing complete")
print("Final shape:", df.shape)
print("Label distribution:", df['label'].value_counts())


✅ Preprocessing complete
Final shape: (110290, 3)
Label distribution: label
0    75447
1    34843
Name: count, dtype: int64


In [7]:
import pandas as pd

# Load the preprocessed file
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Bert/Data/processed_new_articles.csv")

# Keep only text and label
df = df[['text', 'label']]

# Convert labels from float to int
df['label'] = df['label'].astype(int)

# Save clean final dataset
df.to_csv("/content/drive/MyDrive/Colab Notebooks/Bert/Data/final_new_articles.csv", index=False)

print("✅ Final dataset ready")
print(df.head())


✅ Final dataset ready
                                                text  label
0  thousands of patients 'wait months for nhs men...      1
1  peel garlic like magic: this is so easy! we al...      0
2  does your doctor treat disease or care for hea...      1
3  big pharma: covid19 vaccine by this fall not h...      0
4  roche ceo: covid19 tests not worth anything tw...      0
